# #3. Granger-Causality-Test

In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
from tqdm.notebook import tqdm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
tqdm.pandas()

In [2]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM
from statsmodels.tsa.vector_ar.vecm import coint_johansen, select_coint_rank
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import grangercausalitytests

# 0. Data Load

## 0-1. qty, cnt 등

In [4]:
final_data = pd.read_csv('최종데이터/final_categorical_0622.csv', encoding='cp949')
adf_corr = pd.read_csv('최종데이터/adf_corr_group_0617.csv', encoding='cp949')
final_data['date'] = final_data['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
final_data = final_data.merge(adf_corr[['sm_cat', 'group']], on='sm_cat')

In [5]:
# sh_data : [정상, high corr]
# ush_data : [비정상, high corr]
# sl_data : [정상, low corr]
# usl_data : [비정상, low corr]

sh_data = final_data[final_data.group=='정상_high']
ush_data = final_data[final_data.group=='비정상_high']
sl_data = final_data[final_data.group=='정상_low']
usl_data = final_data[final_data.group=='비정상_low']

# 정상 데이터 => VAR
# 비정상 데이터 => 공적분 관계가 존재하면 VECM
sh = sh_data[['date', 'age', 'sex', 'sm_cat', 'qty']].groupby(['date', 'sm_cat']).sum().reset_index()
sh = pd.pivot_table(sh, values='qty', index='date', columns='sm_cat').fillna(0)
ush = ush_data[['date', 'age', 'sex', 'sm_cat', 'qty']].groupby(['date', 'sm_cat']).sum().reset_index()
ush = pd.pivot_table(ush, values='qty', index='date', columns='sm_cat').fillna(0)

# qty와 상관관계가 높은 데이터만 사용
sh_item = sh.columns # sm_cat
ush_item = ush.columns # sm_cat

In [7]:
sh.head()

sm_cat,가스온수기,가자미,감/홍시,감말랭이,갓김치,건강즙/녹용,건어물 멸치,건어물 황태,건자두,견과류,...,풋패치,한우육,해조류 다시마,해조류 미역,헤어스타일링용 펌제,헤어케어세트,혼합견과,홍어,화장 비누,회
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.0,21.0,50.0,27.0,26.0,70.0,60.0,47.0,37.0,256.0,...,29.0,169.0,21.0,50.0,38.0,40.0,70.0,7.0,216.0,847.0
2018-01-02,0.0,16.0,60.0,32.0,35.0,115.0,70.0,81.0,36.0,247.0,...,46.0,205.0,30.0,74.0,95.0,45.0,96.0,5.0,343.0,974.0
2018-01-03,0.0,17.0,51.0,28.0,19.0,84.0,88.0,56.0,37.0,165.0,...,39.0,209.0,20.0,64.0,94.0,45.0,107.0,7.0,332.0,1287.0
2018-01-04,0.0,12.0,58.0,30.0,28.0,135.0,89.0,57.0,34.0,186.0,...,46.0,144.0,30.0,60.0,61.0,30.0,75.0,20.0,301.0,1424.0
2018-01-05,0.0,24.0,48.0,23.0,21.0,82.0,64.0,41.0,31.0,160.0,...,28.0,138.0,30.0,60.0,64.0,28.0,67.0,4.0,202.0,1340.0


## 0-2. weather

In [8]:
var_list = ['date', 'avg_ta', 'max_ta', 'min_ta', 'avg_ws', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa']
weather = sh_data.groupby(['date']).mean().reset_index()[var_list]
weather.index = weather['date']
weather = weather.drop('date', axis=1)

# 1. Granger Causality Test

* 두 시계열 변수 모두 정상이면 → VAR 모델 적합 후 그래인저 인과관계 도출  
* 두 시계열 변수 중 하나라도 비정상이면 → 공적분 검정 → 유의한 공적분 관계가 존재하면 VECM 모델 적합 후 인과관계 도출 → 유의한 공적분 관계가 존재하지 않으면 데이터 차분. VAR 모델 적합 후 인과관계 도출   

## 1-1. 정상 상품

In [9]:
st_weather = ['avg_ws', 'PM10', 'PM25', 'sum_ss_hr'] # 정상 시계열 날씨
nst_weather = ['avg_ta', 'max_ta', 'min_ta', 'hm_max', 'max_pa'] # 비정상 시계열 날씨

### 1-1-1. 정상 상품과 정상 날씨

In [10]:
# 최적 시차에서 VAR 모델링 후, 그래인저 인과관계 도출하는 함수 

def VAR_granger(data, weather, sm_cat, w_var):
    x = pd.concat([weather[w_var], data[sm_cat]], axis=1)
    forecasting_model = VAR(x)
    results_aic = []
    for p in range(1,10): # 최적 시차 찾기
        results = forecasting_model.fit(p)
        results_aic.append(results.aic)
    model = VAR(x)
    lag = np.argmin(results_aic) + 1
    var_res = model.fit(lag)
    g_res = var_res.test_causality(caused=sm_cat, causing=w_var)
    if g_res.pvalue <= 0.05:
        reject = 'causality'
    else:
        reject = 'None'
    return pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'var_lag':lag, 'granger_reject':reject}])

In [ ]:
# ===== ADF 확인 =====
adf_sh = pd.DataFrame()

def adf_test(data, sm_cat):
    item = data[sm_cat]
    result = adfuller(item.values)
    if result[1] <= 0.05:
        test_res = '정상' # 기각
    else:
        test_res = '비정상' # 기각하지 못함 
    return pd.DataFrame([{'sm_cat':sm_cat, 'ADF_stats':result[0], 'p-value':result[1], 'test_res':test_res}])

for sm_cat in sh_item:
    adf_sh = pd.concat([adf_sh, adf_test(sh, sm_cat)], axis=0)

In [ ]:
warnings.filterwarnings('ignore')
sh_var_res = pd.DataFrame()

for w in st_weather:
    for sm_cat in sh_item:
        result = VAR_granger(sh, weather, sm_cat, w)
        sh_var_res = pd.concat([sh_var_res, result], axis=0)

In [ ]:
for w in st_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(sh_var_res.loc[sh_var_res.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ws =====
None         117
causality     13
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM10 =====
None         105
causality     25
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM25 =====
None         111
causality     19
Name: granger_reject, dtype: int64
 
===== Granger Causality : sum_ss_hr =====
None         115
causality     15
Name: granger_reject, dtype: int64
 


In [ ]:
sh_var_pivot = pd.pivot_table(sh_var_res, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
sh_var_pivot

weather,PM10,PM25,avg_ws,sum_ss_hr
sm_cat,,,,
가스온수기,causality,None,None,None
가자미,None,None,None,None
감/홍시,None,None,None,None
감말랭이,None,None,None,causality
갓김치,causality,None,None,None
건강즙/녹용,None,None,None,None
건어물 멸치,None,None,None,None
건어물 황태,None,None,None,None
건자두,None,None,None,None


### 1-1-2. 정상 상품과 비정상 날씨

**공적분(Cointegration) 관계**
* 두 비정상적 시계열 x, y의 결합에 의해 생기는 잔차 e가 정상이면, 유의한 공적분 관계가 존재함을 의미  
* x, y간 장기적인 균형관계 (장기적으로 봤을 때 서로 영향을 적당히 미친다). 공적분 관계가 성립함을 확인한 후, VECM으로 예측  
* rank=0 : p-vector의 경우 p개의 단위근을 가지는 불안정한 시계열 = 비정상 시계열

#### 공적분(Cointegration) 검정

In [ ]:
# 공적분 검정 함수
def co_test(data, weather, sm_cat, w_var):
    score, p_value, _ = coint(data[sm_cat], weather[w_var]) # coint(y, x)
    if p_value <= 0.05:
        test_res = '공적분' # 장기적으로 유의한 관계가 있다
    else:
        test_res = 'None' # 장기적으로 유의한 관계가 없다
    return pd.DataFrame([{'sm_cat':sm_cat, w_var:test_res}])

In [ ]:
sh_co_res = pd.DataFrame()
for i in range(len(nst_weather)):
    w_sample = weather[nst_weather[i]]
    res = pd.DataFrame()
    for sm_cat in sh_item:
        res = pd.concat([res, co_test(sh, weather, sm_cat, nst_weather[i])], axis=0)
    if i == 0:
        sh_co_res = pd.concat([sh_co_res, res], axis=1)
    else:
        sh_co_res = pd.concat([sh_co_res, res.drop(['sm_cat'], axis=1)], axis=1) # 정상qty와 비정상날씨 간 공적분이 존재할 경우, vecm 적합 가능

In [ ]:
sh_co_res # 정상qty와 비정상날씨 간 공적분이 존재할 경우, vecm 적합 가능

,sm_cat,avg_ta,max_ta,min_ta,hm_max,max_pa
0,가스온수기,공적분,공적분,공적분,공적분,공적분
0,가자미,공적분,공적분,공적분,공적분,공적분
0,감/홍시,공적분,공적분,공적분,공적분,공적분
0,감말랭이,공적분,공적분,공적분,공적분,공적분
0,갓김치,공적분,공적분,공적분,공적분,공적분
0,건강즙/녹용,공적분,공적분,공적분,공적분,공적분
0,건어물 멸치,공적분,공적분,공적분,공적분,공적분
0,건어물 황태,공적분,공적분,공적분,공적분,공적분
0,건자두,공적분,공적분,공적분,공적분,공적분
0,견과류,공적분,공적분,공적분,공적분,공적분


#### VECM & Granger causality Test

In [ ]:
# print된 결과를 변수에 저장하는 함수

from io import StringIO 
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout
        
# VECM 모델링 후, 그래인저 인과관계를 도출하는 함수
def VECM_granger(data, weather, sm_cat, w_var):
    x = pd.concat([weather[w_var], data[sm_cat]], axis=1)
    vec_rank = select_coint_rank(x, det_order = -1, k_ar_diff = 2, method = 'trace', signif=0.01).rank
    model = VECM(x, k_ar_diff=2, coint_rank=vec_rank)
    vecm_res = model.fit()
    g_res = vecm_res.test_granger_causality(caused=sm_cat, causing=w_var)
    with Capturing() as output:
        print(g_res.summary()[1][2])
    if float(output[0]) <= 0.05:
        reject = 'causality'
    else:
        reject = 'None'
    return pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'granger_reject':reject}])

In [ ]:
# 그래인저 인과관계 검정
sh_vecm_res = pd.DataFrame()

for w_var in nst_weather:
    for sm_cat in sh_item:
        # 공적분 관계가 유의할 때, 그래인저 인과관계 분석
        if (sh_co_res.loc[sh_co_res.sm_cat==sm_cat, w_var] == '공적분').values: 
            result = VECM_granger(sh, weather, sm_cat, w_var)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'granger_reject':'None'}])
        sh_vecm_res = pd.concat([sh_vecm_res, result], axis=0)

In [ ]:
sh_vecm_res

,sm_cat,weather,granger_reject
0,가스온수기,avg_ta,None
0,가자미,avg_ta,causality
0,감/홍시,avg_ta,None
0,감말랭이,avg_ta,None
0,갓김치,avg_ta,causality
...,...,...,...
0,헤어케어세트,max_pa,None
0,혼합견과,max_pa,None
0,홍어,max_pa,None
0,화장 비누,max_pa,None


In [ ]:
for w in nst_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(sh_vecm_res.loc[sh_vecm_res.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ta =====
None         101
causality     29
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_ta =====
None         96
causality    34
Name: granger_reject, dtype: int64
 
===== Granger Causality : min_ta =====
None         117
causality     13
Name: granger_reject, dtype: int64
 
===== Granger Causality : hm_max =====
None         100
causality     30
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_pa =====
None         118
causality     12
Name: granger_reject, dtype: int64
 


In [ ]:
sh_vecm_pivot = pd.pivot_table(sh_vecm_res, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
sh_vecm_pivot

weather,avg_ta,hm_max,max_pa,max_ta,min_ta
sm_cat,,,,,
가스온수기,None,None,None,None,None
가자미,causality,None,None,None,None
감/홍시,None,None,None,None,None
감말랭이,None,None,None,None,None
갓김치,causality,None,None,causality,causality
건강즙/녹용,None,causality,None,None,None
건어물 멸치,None,None,causality,None,None
건어물 황태,None,causality,None,None,causality
건자두,None,None,causality,None,None


### 1-1-3. 정상 상품 최종 인과관계

In [ ]:
sh_granger = pd.concat([sh_vecm_pivot, sh_var_pivot], axis=1).reset_index()
sh_granger['total'] = np.sum(sh_granger.iloc[:,1:]=='causality', axis=1) # 인과관계가 있는 feature 개수

sh_granger.head()

weather,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total
0,가스온수기,None,None,None,None,None,causality,None,None,None,1
1,가자미,causality,None,None,None,None,None,None,None,None,1
2,감/홍시,None,None,None,None,None,None,None,None,None,0
3,감말랭이,None,None,None,None,None,None,None,None,causality,1
4,갓김치,causality,None,None,causality,causality,causality,None,None,None,4


In [ ]:
print('인과관계가 하나라도 있는 상품의 개수 : {}'.format(len(sh_granger[sh_granger.total>0].sm_cat)))
print('인과관계가 하나도 없는 상품의 개수 : {}'.format(len(sh_granger[sh_granger.total==0].sm_cat)))

인과관계가 하나라도 있는 상품의 개수 : 89
인과관계가 하나도 없는 상품의 개수 : 41


In [ ]:
sh_granger[sh_granger.total>0].sm_cat.unique() # 인과관계가 하나라도 있는 상품

array(['가스온수기', '가자미', '감말랭이', '갓김치', '건강즙/녹용', '건어물 멸치', '건어물 황태', '건자두',
       '견과류', '견과류 땅콩', '견과류 마카다미아', '견과류 잣/은행', '견과류 호두', '공기정화 용품',
       '공기청정기', '과채 음료/주스', '기능성 링클케어 화장품', '기능성 영양보습 화장품',
       '기초 화장용 오일/앰플', '기타 농산물', '기타 한방/환제품', '나물', '남성 스킨', '남성 크림',
       '남성 클렌징', '네일리무버', '네일세트', '네일아트', '네일컬러', '녹차', '닭 양념육', '더치커피',
       '돈풍기', '랍스타', '레몬/자몽', '린스', '멀티형 에어컨', '메이크업 박스', '무/배추',
       '바나나/파인애플/망고', '바디 세트', '반건조고구마', '배/포도/과일즙', '백김치',
       '베이스 메이크업용 BB크림', '베이스 메이크업용 CC크림', '베이스 메이크업용 메이크업베이스',
       '벽걸이형 냉온풍기', '보일러', '뷰티 속눈썹/쌍꺼풀', '뷰티용 기름종이', '뷰티용 뷰러',
       '뷰티용 여드름압출기', '삼치', '색조 메이크업 립글로스', '색조 메이크업 마스카라', '색조 메이크업 볼터치',
       '색조 메이크업 쉐딩/하이라이터', '생수', '샤워코롱', '선크림', '소고기 등심/안심', '수산 생물',
       '스킨케어 코팩', '아몬드유/코코넛밀크', '에이드', '옻/칡/쑥즙', '의류건조기', '이온음료',
       '인스턴트커피', '전복 생물', '절임배추/김치속', '젓갈류', '제습기', '차/곡물 음료', '천장형 에어컨',
       '카페 푸드', '클렌징 크림', '클렌징 폼', '태닝용 선크림', '팩도구', '표고버섯', '풋워시/스크럽',
       '풋크림', '해조류 다시마', '헤어스타일링용 펌제', '헤어케

## 1-2. 비정상 상품

### 1-2-1. 비정상 상품과 정상 날씨

#### 공적분 검정

In [ ]:
ush_co_res = pd.DataFrame()
for i in range(len(st_weather)):
    w_sample = weather[st_weather[i]]
    res = pd.DataFrame()
    for sm_cat in ush_item:
        res = pd.concat([res, co_test(ush, weather, sm_cat, st_weather[i])], axis=0)
    if i == 0:
        ush_co_res = pd.concat([ush_co_res, res], axis=1)
    else:
        ush_co_res = pd.concat([ush_co_res, res.drop(['sm_cat'], axis=1)], axis=1) # 비정상qty와 정상날씨 간 공적분이 존재할 경우, vecm 적합 가능

In [ ]:
ush_co_res # 정상qty와 비정상날씨 간 공적분이 존재할 경우, vecm 적합 가능

,sm_cat,avg_ws,PM10,PM25,sum_ss_hr
0,가열식 가습기,None,None,None,None
0,갈비/찜/바비큐용 돈육,공적분,None,공적분,None
0,감귤/한라봉/오렌지,None,None,None,None
0,감자,None,None,None,None
0,건대추,None,None,None,None
0,건어물 진미채,None,None,None,None
0,건포도,None,None,None,None
0,게장류,None,None,None,None
0,견과류 밤,None,None,None,None
0,고추/피망/파프리카,None,None,None,None


#### VECM & Granger Causality Test

In [ ]:
ush_vecm_res = pd.DataFrame()

for w_var in st_weather:
    for sm_cat in ush_item:
        # 공적분 관계가 유의할 때, 그래인저 인과관계 분석
        if (ush_co_res.loc[ush_co_res.sm_cat==sm_cat, w_var] == '공적분').values: 
            result = VECM_granger(ush, weather, sm_cat, w_var)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'granger_reject':'None'}])
        ush_vecm_res = pd.concat([ush_vecm_res, result], axis=0)

In [ ]:
for w in st_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_vecm_res.loc[ush_vecm_res.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ws =====
None    134
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM10 =====
None         132
causality      2
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM25 =====
None         133
causality      1
Name: granger_reject, dtype: int64
 
===== Granger Causality : sum_ss_hr =====
None    134
Name: granger_reject, dtype: int64
 


In [ ]:
# 인과관계가 존재하는 경우
ush_vecm_pivot = pd.pivot_table(ush_vecm_res, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_vecm_pivot

weather,PM10,PM25,avg_ws,sum_ss_hr
sm_cat,,,,
가열식 가습기,None,None,None,None
갈비/찜/바비큐용 돈육,None,None,None,None
감귤/한라봉/오렌지,None,None,None,None
감자,None,None,None,None
건대추,None,None,None,None
건어물 진미채,None,None,None,None
건포도,None,None,None,None
게장류,None,None,None,None
견과류 밤,None,None,None,None


### 1-2-2. 비정상 상품과 비정상 날씨

#### 공적분 검정

In [ ]:
ush_co_res2 = pd.DataFrame()
for i in range(len(nst_weather)):
    w_sample = weather[nst_weather[i]]
    res = pd.DataFrame()
    for sm_cat in ush_item:
        res = pd.concat([res, co_test(ush, weather, sm_cat, nst_weather[i])], axis=0)
    if i == 0:
        ush_co_res2 = pd.concat([ush_co_res2, res], axis=1)
    else:
        ush_co_res2 = pd.concat([ush_co_res2, res.drop(['sm_cat'], axis=1)], axis=1)

In [ ]:
ush_co_res2

,sm_cat,avg_ta,max_ta,min_ta,hm_max,max_pa
0,가열식 가습기,None,None,None,None,None
0,갈비/찜/바비큐용 돈육,None,None,None,None,공적분
0,감귤/한라봉/오렌지,공적분,공적분,None,None,None
0,감자,None,None,None,None,None
0,건대추,None,None,None,None,None
0,건어물 진미채,None,None,None,None,None
0,건포도,공적분,None,공적분,None,None
0,게장류,None,None,None,None,None
0,견과류 밤,None,None,None,None,None
0,고추/피망/파프리카,None,None,None,None,None


#### VECM & Granger Causality Test

In [ ]:
warnings.filterwarnings('ignore')
ush_vecm_res2 = pd.DataFrame()

for w_var in nst_weather:
    for sm_cat in ush_item:
        # 공적분 관계가 유의할 때, 그래인저 인과관계 분석
        if (ush_co_res2.loc[ush_co_res2.sm_cat==sm_cat, w_var] == '공적분').values: 
            result = VECM_granger(ush, weather, sm_cat, w_var)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'granger_reject':'None'}])
        ush_vecm_res2 = pd.concat([ush_vecm_res2, result], axis=0)

In [ ]:
for w in nst_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_vecm_res2.loc[ush_vecm_res2.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ta =====
None         131
causality      3
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_ta =====
None         131
causality      3
Name: granger_reject, dtype: int64
 
===== Granger Causality : min_ta =====
None    134
Name: granger_reject, dtype: int64
 
===== Granger Causality : hm_max =====
None         133
causality      1
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_pa =====
None    134
Name: granger_reject, dtype: int64
 


In [ ]:
ush_vecm_pivot2 = pd.pivot_table(ush_vecm_res2, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_vecm_pivot2

weather,avg_ta,hm_max,max_pa,max_ta,min_ta
sm_cat,,,,,
가열식 가습기,None,None,None,None,None
갈비/찜/바비큐용 돈육,None,None,None,None,None
감귤/한라봉/오렌지,None,None,None,None,None
감자,None,None,None,None,None
건대추,None,None,None,None,None
건어물 진미채,None,None,None,None,None
건포도,None,None,None,None,None
게장류,None,None,None,None,None
견과류 밤,None,None,None,None,None


### 1-2-3. 비정상 상품 인과관계 (with VECM)

In [ ]:
ush_granger1 = pd.concat([ush_vecm_pivot, ush_vecm_pivot2], axis=1).reset_index()
ush_granger1['total'] = np.sum(ush_granger1.iloc[:,1:]=='causality', axis=1) # 인과관계가 있는 feature 개수
ush_granger1 = ush_granger1.loc[ush_granger1.total>0,:]

ush_granger1 # 5개 실화

weather,sm_cat,PM10,PM25,avg_ws,sum_ss_hr,avg_ta,hm_max,max_pa,max_ta,min_ta,total
49,벽걸이 에어컨,None,None,None,None,causality,None,None,causality,None,2
78,에어컨 리모컨,None,None,None,None,causality,None,None,causality,None,2
88,음용 식초,None,None,None,None,causality,None,None,causality,None,2
90,입욕제,causality,causality,None,None,None,causality,None,None,None,3
123,허브차,causality,None,None,None,None,None,None,None,None,1


## 1-3. 비정상 상품 (with VAR)

In [ ]:
ush_item_var = ush_item.copy()
for sm_cat in ush_granger.sm_cat:
    ush_item_var.remove(sm_cat) # VECM을 통해 인과관계를 도출할 수 없었던 sm_cat

### 1-3-1. 비정상 상품과 정상 날씨

In [ ]:
ush_var_res = pd.DataFrame()

for w in st_weather:
    for sm_cat in ush_item_var:
        if (adf_test(ush, sm_cat).test_res=='정상').values:
            result = VAR_granger(ush, weather, sm_cat, w)
        elif (adf_test(weather.diff(1).fillna(0), w).test_res=='정상').values: # 원데이터가 정상이 아니면 1차차분. 1차차분 시 정상이면 VAR 적합
            weather_diff = weather.diff(1).fillna(0)
            result = VAR_granger(ush, weather_diff, sm_cat, w)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w, 'var_lag':lag, 'granger_reject':'None'}])
        ush_var_res = pd.concat([ush_var_res, result], axis=0)

In [ ]:
for w in st_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_var_res.loc[ush_var_res.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ws =====
None         116
causality     13
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM10 =====
None         115
causality     14
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM25 =====
None         119
causality     10
Name: granger_reject, dtype: int64
 
===== Granger Causality : sum_ss_hr =====
None         119
causality     10
Name: granger_reject, dtype: int64
 


In [ ]:
ush_var_pivot = pd.pivot_table(ush_var_res, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_var_pivot

weather,PM10,PM25,avg_ws,sum_ss_hr
sm_cat,,,,
가열식 가습기,causality,causality,causality,None
갈비/찜/바비큐용 돈육,None,None,None,None
감귤/한라봉/오렌지,None,None,None,None
감자,None,None,None,None
건대추,None,None,causality,None
건어물 진미채,causality,None,None,None
건포도,None,None,None,None
게장류,None,None,None,None
견과류 밤,None,causality,None,None


### 1-3-2. 비정상 상품과 비정상 날씨

In [ ]:
ush_var_res2 = pd.DataFrame()

for w in nst_weather:
    for sm_cat in ush_item_var:
        if (adf_test(ush, sm_cat).test_res=='정상').values:
            result = VAR_granger(ush, weather, sm_cat, w)
        elif (adf_test(weather.diff(1).fillna(0), w).test_res=='정상').values:
            weather_diff = weather.diff(1).fillna(0)
            result = VAR_granger(ush, weather_diff, sm_cat, w)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w, 'var_lag':lag, 'granger_reject':'None'}])
        ush_var_res2 = pd.concat([ush_var_res2, result], axis=0)

In [ ]:
for w in nst_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_var_res2.loc[ush_var_res2.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ta =====
None         90
causality    39
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_ta =====
None         99
causality    30
Name: granger_reject, dtype: int64
 
===== Granger Causality : min_ta =====
None         96
causality    33
Name: granger_reject, dtype: int64
 
===== Granger Causality : hm_max =====
None         96
causality    33
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_pa =====
None         119
causality     10
Name: granger_reject, dtype: int64
 


In [ ]:
ush_var_pivot2 = pd.pivot_table(ush_var_res2, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_var_pivot2

weather,avg_ta,hm_max,max_pa,max_ta,min_ta
sm_cat,,,,,
가열식 가습기,causality,causality,None,causality,causality
갈비/찜/바비큐용 돈육,None,None,None,None,None
감귤/한라봉/오렌지,causality,causality,None,None,causality
감자,None,None,None,None,None
건대추,None,None,None,None,causality
건어물 진미채,None,None,None,None,None
건포도,None,None,None,None,None
게장류,None,None,None,None,None
견과류 밤,None,None,None,None,None


### 1-3-3. 비정상 상품 인과관계 (with VAR)

In [ ]:
ush_granger2 = pd.concat([ush_var_pivot, ush_var_pivot2], axis=1).reset_index()
ush_granger2['total'] = np.sum(ush_granger2.iloc[:,1:]=='causality', axis=1) # 인과관계가 있는 feature 개수

ush_granger2 

weather,sm_cat,PM10,PM25,avg_ws,sum_ss_hr,avg_ta,hm_max,max_pa,max_ta,min_ta,total
0,가열식 가습기,causality,causality,causality,None,causality,causality,None,causality,causality,7
1,갈비/찜/바비큐용 돈육,None,None,None,None,None,None,None,None,None,0
2,감귤/한라봉/오렌지,None,None,None,None,causality,causality,None,None,causality,3
3,감자,None,None,None,None,None,None,None,None,None,0
4,건대추,None,None,causality,None,None,None,None,None,causality,2
5,건어물 진미채,causality,None,None,None,None,None,None,None,None,1
6,건포도,None,None,None,None,None,None,None,None,None,0
7,게장류,None,None,None,None,None,None,None,None,None,0
8,견과류 밤,None,causality,None,None,None,None,None,None,None,1
9,고추/피망/파프리카,None,None,None,None,None,None,None,None,None,0


## 1-4. 비정상 상품 최종 인과관계

* VECM 결과와 VAR 결과 병합

In [ ]:
ush_granger = pd.concat([ush_granger1, ush_granger2], axis=0)
ush_granger.head()

weather,sm_cat,PM10,PM25,avg_ws,sum_ss_hr,avg_ta,hm_max,max_pa,max_ta,min_ta,total
49,벽걸이 에어컨,None,None,None,None,causality,None,None,causality,None,2
78,에어컨 리모컨,None,None,None,None,causality,None,None,causality,None,2
88,음용 식초,None,None,None,None,causality,None,None,causality,None,2
90,입욕제,causality,causality,None,None,None,causality,None,None,None,3
123,허브차,causality,None,None,None,None,None,None,None,None,1


In [ ]:
print('인과관계가 하나라도 있는 상품의 개수 : {}'.format(len(ush_granger[ush_granger.total>0].sm_cat)))
print('인과관계가 하나도 없는 상품의 개수 : {}'.format(len(ush_granger[ush_granger.total==0].sm_cat)))

인과관계가 하나라도 있는 상품의 개수 : 77
인과관계가 하나도 없는 상품의 개수 : 57


In [ ]:
ush_granger[ush_granger.total>0].sm_cat.unique() # 인과관계가 하나라도 있는 상품

array(['벽걸이 에어컨', '에어컨 리모컨', '음용 식초', '입욕제', '허브차', '가열식 가습기',
       '감귤/한라봉/오렌지', '건대추', '건어물 진미채', '견과류 밤', '곡물차', '기능성 아이케어 화장품',
       '기능성 화장품 세트', '기초 화장용 로션', '기초 화장용 미스트', '기초 화장용 크림', '기타 주스류',
       '난방용 열풍기', '남성 세트', '남성 쉐이빙', '느타리버섯', '데오드란트', '둥굴레차',
       '딸기/복분자/블루베리', '라디에이터', '루테인/눈 영양제', '명태/동태', '바디 보습제', '바디 클렌져',
       '바디케어용 땀패드', '바디케어용 제모제', '보리차', '복합식 가습기', '부추', '브로콜리/셀러리',
       '비타민', '색조 메이크업 립밤', '색조 메이크업 아이섀도우', '선스프레이', '선케어용 선밤', '소고기 육회',
       '쌈채소', '양배추/양상추', '어린이영양제', '에어워셔', '오메가3/스쿠알렌 영양제', '온수매트',
       '온열매트', '온풍기', '요거트/발효유', '유자차', '율무차', '자연식 가습기', '잡곡 씨드류', '장어',
       '전기온수기', '전기장판', '초음파식 가습기', '초코우유', '카페트매트', '칼슘/철분 영양제',
       '캡슐/POD커피', '컨벡터', '코코아/핫초코', '콜라겐/코큐텐 영양제', '클렌징 로션',
       '탁상/USB 선풍기', '탄산수', '파/양파', '파김치', '포도/거봉/체리', '프로폴리스/로얄젤리',
       '해초류 ', '핸드크림', '황토매트', '휴대용 선풍기', '히터'], dtype=object)

# 2. 데이터 저장

In [ ]:
sh_granger.to_csv('최종데이터/stationary_causality.csv', encoding='cp949', index=None)
ush_granger.to_csv('최종데이터/nonstationary_causality.csv', encoding='cp949', index=None)